# Bite 313. Alternative constructors

In this Bite your are provided with a Domain class and a DomainException custom exception class.

You will add some validation to the current constructor to check if a valid domain name is passed in.

Next you will add a __str__ special method to represent the object (basically the name attribute) and you will write two classmethods to construct domains:

    1. from a URL

    2. from an email

Here you can see the code in action (also make sure you check out the tests):

In [50]:
import re

SITE_PROTOCOL = [
    'http://',
    'https://',
    'https:/'
]

class DomainException(Exception):
    """Raised when an invalid is created."""


class Domain:

    def __init__(self, name):
        # validate a current domain (r'.*\.[a-z]{2,3}$' is fine)
        # if not valid, raise a DomainException
        self.pattern = r'.*\.[a-z]{2,3}$'
        self.pattern_matches = re.findall(self.pattern, name)
        
        if not self.pattern_matches:
            raise DomainException
            
        self.name = name
        
    def __str__(self):
        
        return self.pattern_matches[0]
    
    @classmethod
    def parse_url(cls, url):
        
        # identify and replace protocols for empty char
        for protocol in SITE_PROTOCOL:
            
            url = url.replace(protocol, '')
        
        # Ignore if url have additional endpoints information
        url = url.split(r'/')[0]
        
        # Guarantee to get only two terms to define the domain.
        # Example: www.gmail.com will return gmail.com
        domain = '.'.join(url.split('.')[-2:])
        
        return cls(domain)
    
    @classmethod
    def parse_email(cls, email):
        
        domain = email.split('@')[-1]
        
        return cls(domain)

In [51]:
domain = Domain.parse_url('https:/pybit.es/get-python-source.html-pybit.es')

# Considerations about my solution

1) I decided to create a global list to put terms I want to identify and replace by empty char. I think it's not a problem keep protocols possibilities here because I can enrich this list when I want without impact the code.

2) To parse url, I just removed endpoints info, to keep only the domain and used a join method to concatenate only the two last terms.

3) To parse email was easy, just splitted on '@'

# PyBites Solution

In [62]:
import re


class DomainException(Exception):
    """Raised when an invalid is created."""


class Domain:

    def __init__(self, name):
        if not re.match(r'.*\.[a-z]{2,3}$', name.lower()):
            raise DomainException(f"{name} is an invalid domain")
        self.name = name

    def __str__(self):
        return self.name

    @classmethod
    def parse_url(cls, url):
        name = re.sub(r'https?://(?:www.)?([^.]+?\.[a-z]{2,3}).*',
                      r'\1', url)
        return cls(name)

    @classmethod
    def parse_email(cls, email):
        name = email.split('@')[1]
        return cls(name)

1) use a lot of regular expression to identify the url domain (to parse url)

2) point for me: implemented parse_email in the same way.

# Learning more about tests

In [63]:
import pytest
# from constructors import Domain, DomainException

# To create tests, I just have to create functions with prefix test_ 
# concatenating with a specific name that make sense (clean code =D).

def test_create_domain_from_name():
    domain = Domain('google.com')
    assert str(domain) == 'google.com'
    domain = Domain('nu.nl')
    assert str(domain) == 'nu.nl'
    
def test_invalid_domain():
    with pytest.raises(DomainException):
        Domain('nu.nlll')

# @pytest.mark.parametrize is a tool to run many scenarios with only one test
# parameters:
# string with parameters names separated by ,
# Ex: "arg, expected"
# Second parameter a list of tuples containing many parameters as needed.
# Ex: [('https://pybit.es', 'pybit.es'), ...]
        
@pytest.mark.parametrize("arg, expected", [
    ('https://pybit.es', 'pybit.es'),
    ('http://pybit.es', 'pybit.es'),
    ('https://pybit.es/get-python-source.html', 'pybit.es'),
    ("https://nu.nl", "nu.nl"),
    ("https://python.org/", "python.org"),
    ("https://stackoverflow.com/a/14836456", "stackoverflow.com"),
])
def test_create_domain_from_url(arg, expected):
    domain = Domain.parse_url(arg)
    assert type(domain) == Domain
    assert str(domain) == expected
    
@pytest.mark.parametrize('arg, expected', [
    ('bob@pybit.es', 'pybit.es'),
    ("bob@gmail.com", "gmail.com"),
    ("tim@example.net", "example.net"),
    ("sara@hotmail.co.uk", "hotmail.co.uk"),
])
def test_create_domain_from_email(arg, expected):
    domain = Domain.parse_email(arg)
    assert type(domain) == Domain
    assert str(domain) == expected